# CELDA DEDICADA A LOS IMPORT 

Todos los import han de estar en esta celda

In [19]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import accuracy_score, recall_score, precision_score, confusion_matrix, roc_auc_score, classification_report
from sklearn.inspection import permutation_importance
from sklearn.model_selection import cross_val_predict
from sklearn.feature_selection import SequentialFeatureSelector, mutual_info_classif
import seaborn as sns 

# DECLARACIÓN DE FUNCIONES

Todas las funciones reusables deben ir a continuación.

In [26]:
def print_classification_report(y, y_pred, y_pred_proba):
    matriz_confusion = confusion_matrix(y,y_pred)
    matriz_confusion_df = pd.DataFrame(matriz_confusion, index=['False','True'], columns=['False','True'])
    print(matriz_confusion_df)
    accuracy= accuracy_score(y, y_pred)
    print(" ")
    print(f"Accuracy: {accuracy}")
    precision = precision_score(y, y_pred)
    print(f"Precision: {precision}")
    recall = recall_score(y, y_pred)
    print(f"Recall: {recall}")
    auc = roc_auc_score(y, y_pred_proba)
    print(f"Area bajo la curva: {auc}")

def barplot(input_df, metric_list):
    # Calcular el número de filas necesarias para las subgráficas (en este caso 3 columnas por fila)
    n_rows = int(np.ceil(len(metric_list) / 3))
    n_cols = 3
    fig, axes = plt.subplots(n_rows, n_cols, figsize=(15, 10))
    
    # Aplanar el arreglo de ejes para poder iterar fácilmente
    axes = axes.flatten()
    
    # Crear gráficos de barras por cada columna en metric_list
    for i, col in enumerate(metric_list):
        ax = axes[i]  # Seleccionar el eje correspondiente
        class_counts = input_df[col].value_counts()  # Contar las ocurrencias de cada categoría
        ax.bar(class_counts.index, class_counts.values, color='skyblue')  # Crear gráfico de barras
        ax.set_title(f'Número de Registros por {col}')  # Título del gráfico
        ax.set_xlabel(col)  # Etiqueta del eje X
        ax.set_ylabel('Número de Registros')  # Etiqueta del eje Y
    
    # Eliminar los gráficos sobrantes si hay menos columnas que subgráficas
    for j in range(len(metric_list), len(axes)):
        fig.delaxes(axes[j])
    
    # Ajustar el diseño para que no se solapen las gráficas
    plt.tight_layout()
    plt.show()


def boxplot_grouping_by_cathegorical(input_df, metric_list,output_variable):
    # Configurar el número de filas y columnas de subplots
    n_rows = int(np.ceil(len(metric_list)/3))
    n_cols = 3
    fig, axes = plt.subplots(n_rows, n_cols, figsize=(15, 10))
    
    # Aplanar el arreglo de ejes para iterar fácilmente (si es necesario)
    axes = axes.flatten()
    
    # Crear gráficos de caja por cada columna
    for i, col in enumerate(metric_list):
        ax = axes[i]
        input_df.boxplot(column=[col], by=output_variable, ax=ax, grid=False)
        ax.set_title(f'{col} by Outcome')
        ax.set_xlabel('Outcome')
        ax.set_ylabel(col)
    
    # Quitar gráficos sobrantes si hay menos columnas que subplots
    for j in range(len(metric_list), len(axes)):
        fig.delaxes(axes[j])
    
    # Ajustar el diseño
    plt.tight_layout()
    plt.show()

def apply_one_hot_encoder(input_df, variable_list):
    # Crear el OneHotEncoder
    encoder = OneHotEncoder(sparse=False)

    # Aplicar OneHotEncoder a las columnas seleccionadas
    encoded_data = encoder.fit_transform(input_df[variable_list])

    # Convertir el resultado a un DataFrame para que sea fácil de manejar
    encoded_df = pd.DataFrame(encoded_data, columns=encoder.get_feature_names_out(variable_list))

    # Concatenar las columnas codificadas al DataFrame original (si es necesario)
    input_encoded_df = pd.concat([input_df.drop(columns=variable_list), encoded_df], axis=1)
    
    return input_encoded_df

def fill_nulls_with_value(input_df, variable_list, value):
    output_df = input_df.copy()
    output_df[variable_list] = output_df[variable_list].fillna(value)
    return output_df

def fill_nulls_with_metric(input_df, variable_list, metric):
    output_df = input_df.copy()
    
    # Diccionario para mapear las métricas a funciones pandas
    metric_functions = {
        'mean': lambda x: x.mean(),
        'median': lambda x: x.median(),
        'max': lambda x: x.max(),
        'min': lambda x: x.min(),
    }

    # Verificar si la métrica proporcionada es válida
    if metric not in metric_functions:
        raise ValueError(f"Métrica '{metric}' no es válida. Usa: {list(metric_functions.keys())}")

    # Iterar sobre las columnas de la lista y rellenar los nulos
    for column in variable_list:
        if column in output_df.columns:
            fill_value = metric_functions[metric](output_df[column])  # Calcular el valor de la métrica
            output_df[column] = output_df[column].fillna(fill_value)  # Rellenar valores nulos
        else:
            print(f"Columna '{column}' no encontrada en el DataFrame.")

    return output_df

def barplot_grouping_by_cathegorical(input_df, metric_list,output_variable):
     # Configurar el número de filas y columnas de subplots
    n_rows = int(np.ceil(len(metric_list)/3))
    n_cols = 3
    fig, axes = plt.subplots(n_rows, n_cols, figsize=(15, 10))
    
    # Aplanar el arreglo de ejes para iterar fácilmente (si es necesario)
    axes = axes.flatten()
    
    # Crear gráficos de caja por cada columna
    for i, col in enumerate(metric_list):
        ax = axes[i]
        grouped = input_df.groupby([col, output_variable]).size().unstack(fill_value=0)
        grouped.plot(kind='bar', stacked=True, ax=ax, colormap='Set2')
        ax.set_title(f'{col} by Outcome')
        ax.set_xlabel('Outcome')
        ax.set_ylabel(col)
    
    # Quitar gráficos sobrantes si hay menos columnas que subplots
    for j in range(len(metric_list), len(axes)):
        fig.delaxes(axes[j])
    
    # Ajustar el diseño
    plt.tight_layout()
    plt.show()    

def mdi(X,y):
    rft = RandomForestClassifier(random_state=42)
    rft.fit(X,y)
    mdi_importance = rft.feature_importances_
    
    feature_names = X.columns
    indices = np.argsort(mdi_importance)[::-1]  # Ordenar por importancia
    
    plt.figure(figsize=(10, 6))
    plt.bar(range(len(mdi_importance)), mdi_importance[indices], align="center", color='skyblue')
    plt.xticks(range(len(mdi_importance)), [feature_names[i] for i in indices], rotation=45, ha="right")
    plt.xlabel("Features")
    plt.ylabel("MDI Importance")
    plt.title("Feature Importance (MDI) - Random Forest")
    plt.tight_layout()
    plt.show()

def mda(X,y):
    feature_names = X.columns
    rft = RandomForestClassifier(random_state=42)    
    rft.fit(X, y)
    perm_importance = permutation_importance(rft, X, y, scoring="accuracy", n_repeats=10, random_state=42)
    importance_means = perm_importance.importances_mean
    importance_stds = perm_importance.importances_std
    indices = np.argsort(importance_means)[::-1]  # Ordenar por importancia
    
    plt.figure(figsize=(10, 6))
    plt.bar(range(len(importance_means)), importance_means[indices], yerr=importance_stds[indices], align="center", color="lightcoral", capsize=5)
    plt.xticks(range(len(importance_means)), [feature_names[i] for i in indices], rotation=45, ha="right")
    plt.xlabel("Features")
    plt.ylabel("MDA Importance")
    plt.title("Feature Importance (MDA) - Random Forest")
    plt.tight_layout()
    plt.show()

def plot_histograms(dataset, variables, bins=10):
    n_vars = len(variables)
    n_cols = 3
    n_rows = (n_vars // n_cols) + (n_vars % n_cols > 0)

    fig, axes = plt.subplots(n_rows, n_cols, figsize=(n_cols * 5, n_rows * 4))
    axes = axes.flatten()
    
    for i, var in enumerate(variables):
        axes[i].hist(dataset[var].dropna(), bins=bins, alpha=0.7, color='blue')
        axes[i].set_title(var)
    
    for j in range(i + 1, len(axes)):
        fig.delaxes(axes[j])
    
    plt.tight_layout()
    plt.show()

def plot_mutual_information_heatmap(df, target):
    # Separamos las características (X) y la variable objetivo (y)
    X = df.drop(columns=[target])
    y = df[target]
    
    # Calculamos la información mutua entre las características y la variable objetivo
    mi = mutual_info_classif(X, y)
    
    # Creamos un DataFrame con los resultados
    mi_df = pd.DataFrame(mi, index=X.columns, columns=['Mutual Information'])
    
    # Ordenamos los resultados por la información mutua
    mi_df = mi_df.sort_values(by='Mutual Information', ascending=False)
    
    # Dibujamos el heatmap
    plt.figure(figsize=(10, 6))
    sns.heatmap(mi_df.T, annot=True, cmap="YlGnBu", cbar=True, fmt='.2f', linewidths=0.5)
    plt.title('Mutual Information between Features and Target')
    plt.show()

def plot_correlation_heatmap(df, method = None):
    if method is None:
        # Calculamos la matriz de correlación entre las variables numéricas
        corr_matrix = df.corr()
    else:
        corr_matrix = df.corr(method = method)
    # Dibujamos el heatmap
    plt.figure(figsize=(12, 8))
    sns.heatmap(corr_matrix, annot=True, cmap="coolwarm", fmt='.2f', linewidths=0.5, cbar=True)
    plt.title('Correlación entre las Variables')
    plt.show()

# LECTURA DEL FICHERO

In [ ]:
heart_df = pd.read_csv("heart.csv")

# ANÁLISIS DESCRIPTIVO

## INFO Y HEAD

## DEFINIMOS LAS VARIABLES CATEGÓRICAS/DISCRETAS Y LAS VARIABLES NUMÉRICAS

## DESCRIBE DE LAS VARIABLES NUMÉRICAS

## QUÉ REPRESENTA CADA VARIABLE

age :
sex :
cp :
trestbps :
chol :
fbs :
restecg :
thalach :
exang :
oldpeak :
slope :
ca :
thal :
target :

## DISTRIBUCIÓN DE CADA VARIABLE


## CORRELACIONES

Boxplot agrupando por target para las continuas y barplot agrupando por target para las discretas

Heatmap de correlaciones

## DEFINIMOS EL OBJETIVO/MÉTRICA DEL PROBLEMA


## FEATURE IMPORTANCE

Aplicamos MDI y MDA.